In [ ]:
import os

from dotenv import load_dotenv

from autogen.agentchat.conversable_agent import ConversableAgent
from autogen.tools.md_tools import markdown_convert, markdown_convert_tool

load_dotenv()


# Let's first define the assistant agent that suggests tool calls.
assistant = ConversableAgent(
    name="Assistant",
    system_message="You are a helpful AI assistant. "
    "You can help with converting local files or URLs to markdown. use the provided function markdown_convert. "
    "Return 'TERMINATE' when the task is done.",
    llm_config={"model": "gpt-4o", "api_key": os.environ["API_KEY"]},
)

# The user proxy agent is used for interacting with the assistant agent
# and executes tool calls.
user_proxy = ConversableAgent(
    name="User",
    llm_config=False,
    is_termination_msg=lambda msg: msg.get("content") is not None and "TERMINATE" in msg["content"],
    human_input_mode="NEVER",
)

from autogen import register_function

# Register the calculator function to the two agents.
register_function(
    markdown_convert,
    caller=assistant,  # The assistant agent can suggest calls to the calculator.
    executor=user_proxy,  # The user proxy agent can execute the calculator calls.
    name="markdown_convert",  # By default, the function name is used as the tool name.
    description="Converts a local file or a URL to markdown.",  # A description of the tool.
)

In [ ]:
chat_result = user_proxy.initiate_chat(
    assistant, message="What is the content of the file test/agentchat/contrib/markdown_converter/test_files/test.xlsx"
)